In [ ]:
# Installing packages needed
install.packages("caTools")
install.packages("ROCR")
install.packages("dplyr")

In [ ]:
install.packages("Hmisc")

In [ ]:
library(Hmisc)

In [ ]:
# Loading needed packages 
library(caTools)
library(ROCR)
library(dplyr)

In [6]:
# Importing HEMQ.csv file
# Specify missing values
df <- read.csv("Datasets/HEMQ.csv",na.strings=c("NA",""))

# Initial exploration
summary(df)

      BAD              LOAN          MORTDUE           VALUE       
 Min.   :0.0000   Min.   : 1100   Min.   :  2063   Min.   :  8000  
 1st Qu.:0.0000   1st Qu.:11100   1st Qu.: 46276   1st Qu.: 66076  
 Median :0.0000   Median :16300   Median : 65019   Median : 89236  
 Mean   :0.1995   Mean   :18608   Mean   : 73761   Mean   :101776  
 3rd Qu.:0.0000   3rd Qu.:23300   3rd Qu.: 91488   3rd Qu.:119824  
 Max.   :1.0000   Max.   :89900   Max.   :399550   Max.   :855909  
                                  NA's   :518      NA's   :112     
     REASON          JOB            YOJ             DEROG        
 DebtCon:3928   Mgr    : 767   Min.   : 0.000   Min.   : 0.0000  
 HomeImp:1780   Office : 948   1st Qu.: 3.000   1st Qu.: 0.0000  
 NA's   : 252   Other  :2388   Median : 7.000   Median : 0.0000  
                ProfExe:1276   Mean   : 8.922   Mean   : 0.2546  
                Sales  : 109   3rd Qu.:13.000   3rd Qu.: 0.0000  
                Self   : 193   Max.   :41.000   Max.   :10.0

In [7]:
str(df)

'data.frame':	5960 obs. of  13 variables:
 $ BAD    : int  1 1 1 1 0 1 1 1 1 1 ...
 $ LOAN   : int  1100 1300 1500 1500 1700 1700 1800 1800 2000 2000 ...
 $ MORTDUE: num  25860 70053 13500 NA 97800 ...
 $ VALUE  : num  39025 68400 16700 NA 112000 ...
 $ REASON : Factor w/ 2 levels "DebtCon","HomeImp": 2 2 2 NA 2 2 2 2 2 2 ...
 $ JOB    : Factor w/ 6 levels "Mgr","Office",..: 3 3 3 NA 2 3 3 3 3 5 ...
 $ YOJ    : num  10.5 7 4 NA 3 9 5 11 3 16 ...
 $ DEROG  : int  0 0 0 NA 0 0 3 0 0 0 ...
 $ DELINQ : int  0 2 0 NA 0 0 2 0 2 0 ...
 $ CLAGE  : num  94.4 121.8 149.5 NA 93.3 ...
 $ NINQ   : int  1 0 1 NA 0 1 1 0 1 0 ...
 $ CLNO   : int  9 14 10 NA 14 8 17 8 12 13 ...
 $ DEBTINC: num  NA NA NA NA NA ...


In [8]:
# Change DV data type
df$BAD <- factor(df$BAD)

In [9]:
# Set a random see so your "random" results are the same as me (optional)
set.seed(101)

In [10]:
# Data partition
# Split up the sample, basically randomly assigns a booleans to a new column "sample"
# SplitRatio: percent of sample==TRUE
sample <- sample.split(df$BAD, SplitRatio = 0.7)

In [11]:
# Create Training Data
df.train = subset(df, sample == TRUE)

In [12]:
# Create Validation Data
df.valid = subset(df, sample == FALSE)

In [13]:
# Run a very simple baseline model with the training dataset
baseline.model <- glm(formula=BAD ~ . , family = binomial(link='logit'),data = df.train)
# generalized linear model - logistic model
#. (dot) means all the independent variables will be used 
#logit link function for model
#1 argument - what the model looks like
#2 argument - specify its logistic model
#3 argument - what data are you using

In [14]:
# View the model results
summary(baseline.model)


Call:
glm(formula = BAD ~ ., family = binomial(link = "logit"), data = df.train)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-1.6561  -0.4119  -0.2952  -0.2088   3.2332  

Coefficients:
                Estimate Std. Error z value Pr(>|z|)    
(Intercept)   -4.245e+00  5.546e-01  -7.655 1.94e-14 ***
LOAN          -1.624e-05  9.656e-06  -1.682 0.092557 .  
MORTDUE       -6.278e-06  4.588e-06  -1.368 0.171192    
VALUE          2.962e-06  3.983e-06   0.744 0.457144    
REASONHomeImp -6.863e-02  1.905e-01  -0.360 0.718724    
JOBOffice     -4.510e-01  3.139e-01  -1.437 0.150757    
JOBOther      -5.385e-02  2.463e-01  -0.219 0.826887    
JOBProfExe    -7.965e-02  2.826e-01  -0.282 0.778038    
JOBSales       8.948e-01  5.612e-01   1.595 0.110802    
JOBSelf        8.581e-01  4.622e-01   1.857 0.063374 .  
YOJ           -2.291e-02  1.239e-02  -1.849 0.064525 .  
DEROG          7.953e-01  1.205e-01   6.598 4.16e-11 ***
DELINQ         7.185e-01  8.265e-02   8.693  < 2

In [15]:
#Evaluation model performance using the validation dataset
#Predict the default probabilities based on the validation dataset
pred.probabilities <- predict(baseline.model,newdata=df.valid,type='response')

In [16]:
#Turn the default probabilities to binary
pred.results <- ifelse(pred.probabilities > 0.5,1,0)

In [17]:
# Create the confusion matrix
print("The confusion matrix is:")
print(table(pred.results, df.valid$BAD))

[1] "The confusion matrix is:"
            
pred.results   0   1
           0 922  64
           1   9  19


In [18]:
# Create the ROC curve
pred <- prediction(pred.probabilities,df.valid$BAD)
perf <- performance(pred, "tpr", "fpr")
plot(perf)
abline(a=0,b=1)

In [19]:
# Calculate and print AUC value
auc <- performance(pred, measure="auc")
auc <- auc@y.values[[1]]
print(paste("AUC for the baseline regression model is:", auc))

[1] "AUC for the baseline regression model is: 0.817530055776267"
